In [1]:
import pandas as pd

# Data sources:

https://www.kaggle.com/datasets/shuyangli94/food-com-recipes-and-user-interactions?select=RAW_recipes.csv

https://www.kaggle.com/datasets/shuyangli94/foodcom-recipes-with-search-terms-and-tags

https://www.kaggle.com/datasets/irkaal/foodcom-recipes-and-reviews

# Read data

Download all data sources and unpack then in src directory

In [2]:
# read cvs file "PP_recipes.csv" and print first 10 rows
raw_recipes = pd.read_csv('src/RAW_recipes.csv')
raw_recipes = raw_recipes[['id', 'name', 'minutes', 'submitted', 'tags', 'n_steps', 'steps', 'description']]
raw_recipes.head(2)

,id,name,minutes,submitted,tags,n_steps,steps,description
0,137739,arriba baked winter squash mexican style,55,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...
1,31490,a bit different breakfast pizza,30,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...


In [3]:
recipes_with_ingredients = pd.read_csv('src/recipes_w_search_terms.csv')
recipes_with_ingredients = recipes_with_ingredients[['id', 'ingredients_raw_str', 'ingredients']]
recipes_with_ingredients.head(2)

,id,ingredients_raw_str,ingredients
0,96313,"[""4 cups water"",""1 cup uncooked old f...","['water', 'grits', 'salt', 'cheddar cheese', '..."
1,232037,"[""1 medium onion, chopped coarse "",""1 m...","['onion', 'red bell pepper', 'garlic cloves', ..."


In [4]:
recipes_big = pd.read_parquet('src/recipes_with_imgs.parquet', engine='fastparquet')
recipes_big = recipes_big[['RecipeId', 'AuthorId', 'AuthorName', 'ReviewCount']]
recipes_big.head(2)

,RecipeId,AuthorId,AuthorName,ReviewCount
0,38.0,1533,Dancer,4.0
1,39.0,1567,elly9812,1.0


In [6]:
recipes_tokenized = pd.read_csv('src/PP_recipes.csv')
recipes_tokenized = recipes_tokenized[['id', 'calorie_level', 'ingredient_ids']]
recipes_tokenized.head(2)

,id,calorie_level,ingredient_ids
0,424415,0,"[389, 7655, 6270, 1527, 3406]"
1,146223,0,"[2683, 4969, 800, 5298, 840, 2499, 6632, 7022,..."


# Prepare recipes base

In [7]:
# merge interactions_count with recipes based in id
recipes_full = pd.merge(raw_recipes, recipes_with_ingredients, left_on='id', right_on='id', how='left')
# merge interactions_count with recipes based in id
recipes_full = pd.merge(recipes_full, recipes_tokenized, left_on='id', right_on='id', how='left')
# merge with recipes_big
recipes_full = pd.merge(recipes_full, recipes_big, left_on='id', right_on='RecipeId', how='left')

recipes_full = recipes_full.dropna(subset=['AuthorId', 'AuthorName'])

In [8]:
# sort recipes by review count and take top 1000
base_recipes = recipes_full.sort_values(by=['ReviewCount'], ascending=False)
base_recipes.dropna()

# drop recipes where ingredient_ids are nan
base_recipes = base_recipes.dropna(subset=['ingredient_ids'])

base_recipes = base_recipes.head(5000)

base_recipes

,id,name,minutes,submitted,tags,n_steps,steps,description,ingredients_raw_str,ingredients,calorie_level,ingredient_ids,RecipeId,AuthorId,AuthorName,ReviewCount
22168,2886,best banana bread,65,1999-09-26,"['time-to-make', 'course', 'main-ingredient', ...",13,"['remove odd pots and pans from oven', 'prehea...",you'll never need another banana bread recipe ...,"[""1/2 cup butter, softened "",""1 cup g...","['butter', 'granulated sugar', 'eggs', 'banana...",1.0,"[840, 3355, 2499, 342, 63, 335, 6270, 7449]",2886.0,1762.0,lkadlec,2273.0
213826,27208,to die for crock pot roast,545,2002-05-03,"['weeknight', 'time-to-make', 'course', 'main-...",7,"['place beef roast in crock pot', 'mix the dri...","amazing flavor, and so simple! no salt needed ...","[""1 (4 -5 lb) beef roast, any kind "",""1 (...","['beef roast', 'brown gravy mix', 'dried itali...",1.0,"[469, 784, 2200, 2200, 7655]",27208.0,28201.0,yooper,1692.0
66687,89204,crock pot chicken with black beans cream cheese,243,2004-04-16,"['time-to-make', 'course', 'main-ingredient', ...",5,"['take 4-5 frozen , yes , frozen , boneless ch...",i love this crock-pot chicken recipe for two r...,"[""4 -5 boneless chicken breasts, frozen ...","['boneless chicken breasts', 'black beans', 'c...",2.0,"[1252, 553, 1803, 6265, 1910]",89204.0,137839.0,Jen Santiago,1657.0
62431,39087,creamy cajun chicken pasta,25,2002-09-02,"['30-minutes-or-less', 'time-to-make', 'course...",4,['place chicken and cajun seasoning in a bowl ...,n'awlin's style of chicken with an updated alf...,"[""2 boneless skinless chicken breast hal...","['boneless skinless chicken breast halves', 'l...",2.0,"[1254, 4416, 905, 840, 6335, 3670, 6926, 6270,...",39087.0,30534.0,Lorac,1586.0
22388,67256,best ever banana cake with cream cheese frosting,75,2003-07-24,"['weeknight', 'time-to-make', 'course', 'main-...",18,"['preheat oven to 275f', 'grease and flour a 9...",this is one of (if not) the best banana cake i...,"[""1 1/2 cups bananas, mashed, ripe "",""2 ...","['bananas', 'lemon juice', 'flour', 'baking so...",2.0,"[342, 4253, 2683, 335, 6270, 840, 6906, 2499, ...",67256.0,82367.0,Rhonda J,1409.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123547,442185,light and fluffy biscuits,30,2010-11-16,"['30-minutes-or-less', 'time-to-make', 'course...",11,"['combine the flour , salt , baking powder and...","if you like biscuits, i am sure you will like ...","[""4 cups all-purpose flour"",""1 teaspoon...","['all-purpose flour', 'salt', 'baking powder',...",1.0,"[63, 6270, 332, 6906, 855, 2499, 4717]",442185.0,1728457.0,skeptic777,25.0
92715,348549,glazed carrots with maple syrup and sesame seeds,17,2009-01-09,"['30-minutes-or-less', 'time-to-make', 'course...",4,['put the carrots into a medium pan with half ...,"wanting a quick carrot dish last night, i turn...","[""1 lb carrot, peeled and cut into batons...","['carrot', 'butter', 'honey', 'salt', 'water',...",0.0,"[1093, 840, 3723, 6270, 7655, 6416]",348549.0,323186.0,Karen Elizabeth,25.0
227203,54787,whoopie pies the real deal lancaster co r...,38,2003-02-24,"['60-minutes-or-less', 'time-to-make', 'course...",14,"['cream shortening , sugar and eggs', 'add van...",mmmmmmmmmmmmmmmmm! if you've never had a whoop...,"[""1 cup oil"",""2 cups brown sugar"",""4...","['oil', 'brown sugar', 'eggs', 'flour', 'salt'...",2.0,"[4987, 800, 2499, 2683, 6270, 335, 1584, 4717,...",54787.0,41706.0,WJKing,25.0
34388,17449,campbell s chicken broccoli alfredo,20,2002-01-18,"['30-minutes-or-less', 'time-to-make', 'course...",7,['cook linguine according to package direction...,NaN,"[""8 ounces linguine"",""1 cup fresh, fr...","['linguine', 'broccoli florets', 'butter', 'bo...",2.0,"[4416, 761, 840, 1252, 1920, 4717, 5180, 5319]",17449.0,20754.0,RecipeNut,25.0


# Ingredients table

In [10]:
ingredients = pd.read_pickle('src/ingr_map.pkl')

# create new ingredients table data, that stores only processed and id columns off current ingredients data
ingredients_processed = ingredients[['id', 'replaced']]
ingredients_processed.columns = ['id', 'ingredient_name']

# remove repetitions in ingredients_processed
ingredients_processed = ingredients_processed.drop_duplicates()
# save processed ingredients to csv file
ingredients_processed.to_csv('result/ingredients.csv', index=False)

ingredients_processed

,id,ingredient_name
0,4308,lettuce
42,2744,french vanilla pudding and pie filling mix
43,6843,stove top stuffing mix
45,1910,cream cheese
86,1168,cheddar
...,...,...
11654,6702,soybean
11655,3318,goose
11656,47,ajwain
11657,750,brinjal


# Recipes-ingredients table

In [11]:
import ast
# create empty pandas dataframe
recipes_ingredients = pd.DataFrame()

to_drop = []
for index, row in base_recipes.iterrows():
    try:
        ingredients_raw = ast.literal_eval(row['ingredients_raw_str'])
        ingredients_ids = ast.literal_eval(row['ingredient_ids'])
        ingredients_names = ast.literal_eval(row['ingredients'])
    except:
        to_drop.append(index)
        continue

    if len(ingredients_names) > len(ingredients_ids):
        ingredients_names = ingredients_names[:len(ingredients_ids)]
    elif len(ingredients_names) < len(ingredients_ids):
        ingredients_ids = ingredients_ids[:len(ingredients_names)]

    if len(ingredients_raw) > len(ingredients_names):
        ingredients_raw = ingredients_raw[:len(ingredients_names)]
    elif len(ingredients_raw) < len(ingredients_names):
        ingredients_names = ingredients_names[:len(ingredients_raw)]

    ingredients_quantity = []
    for i, ingredient_raw in enumerate(ingredients_raw):
        words = ingredient_raw.split()
        if len(words) == 0:
            ingredients_quantity.append('')
            continue
        
        quantity = words[0]
        words = words[1:]

        
        if (len(words) > 0):
            if '(' in words[0]:
                while ')' not in words[0]:
                    quantity += ' ' + words[0]
                    words = words[1:]
                quantity += ' ' + words[0]
                words = words[1:]
        
        # add some possible fractions

        if (len(words) > 0):
            if any(char.isdigit() for char in words[0]):
                quantity += ' ' + words[0]
                words = words[1:]

        if (len(words) > 0):
            if '(' in words[0]:
                while ')' not in words[0]:
                    quantity += ' ' + words[0]
                    words = words[1:]
                quantity += ' ' + words[0]
                words = words[1:]
        
        # add measurement unit
        if (len(words) > 0):
            if words[0] != ingredients_names[i].split(' ')[0]:
                quantity += ' ' + words[0]
            
        ingredients_quantity.append(quantity)

    # create dataframe with names and id's 
    ingredients_array = pd.DataFrame({'ingredient_quantity': ingredients_quantity, 'ingredient_id': ingredients_ids})

    # add row id column to tags_array
    ingredients_array['recipe_id'] = row['id']

    # add row to recipes_tags dataframe
    recipes_ingredients = pd.concat([recipes_ingredients, ingredients_array], ignore_index=True)

# drop recipes with index in to_drop
base_recipes = base_recipes.drop(to_drop)

# change the order of columns in recipes_ingredients
recipes_ingredients = recipes_ingredients[['recipe_id', 'ingredient_quantity', 'ingredient_id']]

recipes_ingredients.to_csv('result/recipes_ingredients.csv', index=False)
print(len(to_drop), len(base_recipes))
recipes_ingredients.head(2)

88 4912


,recipe_id,ingredient_quantity,ingredient_id
0,2886,1/2 cup,840
1,2886,1 cup,3355


# Tags table

In [12]:
import ast
# create empty pandas dataframe
tags = pd.DataFrame()

# iterate for every row in recipes_with_interactions_1
for index, row in base_recipes.iterrows():
    tags_array = ast.literal_eval(row['tags'])
    # add tags to tags dataframe
    tags = pd.concat([tags, pd.DataFrame(tags_array)], ignore_index=True)

# remove repetitions in tags
tags = tags.drop_duplicates()
# name first column in tags "tag_name"
tags.columns = ['tag_name']
tags['id'] = tags.index

tags = tags[['id', 'tag_name']]
# add id column to tags

tags.to_csv('result/tags.csv', index=False)
tags.head(2)

,id,tag_name
0,0,time-to-make
1,1,course


# Recipes-tags table

In [13]:
# create empty pandas dataframe
recipes_tags = pd.DataFrame()

# iterate for every row in recipes_with_interactions_1
for index, row in base_recipes.iterrows():
    tags_array = ast.literal_eval(row['tags'])

    # find tag id for every tag name in tags_array
    tags_ids = tags[tags['tag_name'].isin(tags_array)]['id'].values

    tags_ids_df = pd.DataFrame(tags_ids, columns=['tag_id'])

    # add row id column to tags_array
    tags_ids_df['recipe_id'] = row['id']

    # add row to recipes_tags dataframe
    recipes_tags = pd.concat([recipes_tags, tags_ids_df], ignore_index=True)

# change the order of columns in recipes_tags
recipes_tags = recipes_tags[['recipe_id', 'tag_id']]
recipes_tags.to_csv('result/recipes_tags.csv', index=False)

recipes_tags.head(2)

,recipe_id,tag_id
0,2886,0
1,2886,1


# Recipes-steps table

In [14]:
recipes_steps = pd.DataFrame()

for index, row in base_recipes.iterrows():
    steps = ast.literal_eval(row['steps'])
    
    # create dataframe with names and id's 
    steps = pd.DataFrame(steps, columns=['step'])

    # add row id column to tags_array
    steps['recipe_id'] = row['id']

    # add row to recipes_tags dataframe
    recipes_steps = pd.concat([recipes_steps, steps], ignore_index=True)

# change the order of columns in recipes_ingredients
recipes_steps = recipes_steps[['recipe_id', 'step']]

recipes_steps.to_csv('result/recipes_steps.csv', index=False)
recipes_steps

,recipe_id,step
0,2886,remove odd pots and pans from oven
1,2886,preheat oven to 350 / 180
2,2886,cream together butter and sugar
3,2886,add eggs and crushed bananas
4,2886,combine well
...,...,...
43435,26233,stir in everything else and heat to boiling
43436,26233,put in a 1 1 / 2 quart casserole and cover
43437,26233,bake for 45 minutes
43438,26233,remove bay leaves


# Users table

In [15]:
# find distinct contributor_id in recipes_table
users = base_recipes[['AuthorId', 'AuthorName']].drop_duplicates()

# change the name of the first column "id"
users.columns = ['id', 'nickname']

# add email column which is equal to id
users['email'] = users['nickname'].replace(" ", "") + '@unknown.com'
users['password'] = 'password'

users.to_csv('result/users.csv', index=False)
users

,id,nickname,email,password
22168,1762.0,lkadlec,lkadlec@unknown.com,password
213826,28201.0,yooper,yooper@unknown.com,password
66687,137839.0,Jen Santiago,Jen Santiago@unknown.com,password
62431,30534.0,Lorac,Lorac@unknown.com,password
22388,82367.0,Rhonda J,Rhonda J@unknown.com,password
...,...,...,...,...
225193,353491.0,Clean Plate Club,Clean Plate Club@unknown.com,password
99246,47579.0,Peeps,Peeps@unknown.com,password
205436,58552.0,NHNichole,NHNichole@unknown.com,password
6247,135566.0,LainieBug,LainieBug@unknown.com,password


# Recipes table

In [16]:
recipes_table = base_recipes.drop(['steps', 'ingredients', 'ingredient_ids', 'ingredients_raw_str', 'tags', 'RecipeId', 'n_steps', 'AuthorName'], axis=1)

# capitalize descriptions
recipes_table['description'] = recipes_table['description'].str.capitalize()

# change column name call ReviewCount to reactions
recipes_table = recipes_table.rename(columns={'ReviewCount': 'reactions'})

recipes_table.to_csv('result/recipes.csv', index=False)

recipes_table

,id,name,minutes,submitted,description,calorie_level,AuthorId,reactions
22168,2886,best banana bread,65,1999-09-26,You'll never need another banana bread recipe ...,1.0,1762.0,2273.0
213826,27208,to die for crock pot roast,545,2002-05-03,"Amazing flavor, and so simple! no salt needed ...",1.0,28201.0,1692.0
66687,89204,crock pot chicken with black beans cream cheese,243,2004-04-16,I love this crock-pot chicken recipe for two r...,2.0,137839.0,1657.0
62431,39087,creamy cajun chicken pasta,25,2002-09-02,N'awlin's style of chicken with an updated alf...,2.0,30534.0,1586.0
22388,67256,best ever banana cake with cream cheese frosting,75,2003-07-24,This is one of (if not) the best banana cake i...,2.0,82367.0,1409.0
...,...,...,...,...,...,...,...,...
123547,442185,light and fluffy biscuits,30,2010-11-16,"If you like biscuits, i am sure you will like ...",1.0,1728457.0,25.0
92715,348549,glazed carrots with maple syrup and sesame seeds,17,2009-01-09,"Wanting a quick carrot dish last night, i turn...",0.0,323186.0,25.0
227203,54787,whoopie pies the real deal lancaster co r...,38,2003-02-24,Mmmmmmmmmmmmmmmmm! if you've never had a whoop...,2.0,41706.0,25.0
34388,17449,campbell s chicken broccoli alfredo,20,2002-01-18,NaN,2.0,20754.0,25.0


In [36]:
# find distinct contributor_id in recipes_table
import json
import requests
from tqdm import tqdm

users = base_recipes[['AuthorId', 'AuthorName']].drop_duplicates()


# change the name of the first column "id"
users.columns = ['id', 'username']

users['username'] = users['username'].str.replace(" ", "")

# add email column which is equal to id
users['email'] = users['username'] + '@unknown.com'
users['password'] = 'pass' + users['username']

# for each row send post request to /api/users and store the response in users_table
users_table = pd.DataFrame()
for index, row in users.iterrows():
    # change request dat to json string
    request_data = json.dumps({ 'email': row['email'] + "tmp", 'password': row['password'], 'username': row['username'] })
    response = requests.post('http://127.0.0.1:8000/users/', data=request_data, 
    headers={'Content-Type': 'application/json', 'Accept': 'application/json'})
    
    users_table = users_table.append(pd.DataFrame({'local_id': row['id'], 'api_id': response.json().get('id')}, index=[0]))


0it [00:00, ?it/s]/tmp/ipykernel_686168/3556674751.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  users_table = users_table.append(pd.DataFrame({'local_id': row['id'], 'api_id': response.json().get('id')}, index=[0]))
1it [00:00,  2.61it/s]/tmp/ipykernel_686168/3556674751.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  users_table = users_table.append(pd.DataFrame({'local_id': row['id'], 'api_id': response.json().get('id')}, index=[0]))
2it [00:00,  2.31it/s]/tmp/ipykernel_686168/3556674751.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  users_table = users_table.append(pd.DataFrame({'local_id': row['id'], 'api_id': response.json().get('id')}, index=[0]))
3it [00:01,  2.48it/s]/tmp/ipykernel_686168/355667475

KeyboardInterrupt: 

In [35]:
# join users_table with users
users = users.merge(users_table, left_on='id', right_on='local_id')
users

,id,username,email,password,local_id_x,api_id_x,local_id_y,api_id_y,local_id,api_id
0,1762.0,lkadlec,lkadlec@unknown.com,passlkadlec,1762.0,5,1762.0,5,1762.0,5
